# U.S. Geological Survey Class GW3099
Advanced Modeling of Groundwater Flow (GW3099)\
Boise, Idaho\
September 16 - 20, 2024

![title](../../images/ClassLocation.jpg)

# Pywatershed MF6 coupling

## The challenge of interdisciplinary modeling
 
A recurring challenge of water sciences is working across disciplines. Increasingly, this is where important scientific questions and advances are located. From a software perspective, the peril of interdisciplinary work is that it can become  stale over time. This problem requires intentional software design to codes interoperable.

One example of this is the case of GSFLOW, the USGS coupled groundwater-surfacewater flow model (Markstrom et al., 2008). GSFLOW represents a significant advancement in modeling many aspects of combined groundwater and surface water flow. However, over time, the GSFLOW code has diverged from the current and improved versions of both the USGS's hydrologic (Precipitation Runoff Modeling System, PRMS; Regan et al., 2022) and groundwater (MODFLOW, Langevin et al., 2017) models which it initially bridged. 

The challenge of working across-disciplines in a sustainable way is one issue addressed by the development of the Basic Model Interface (BMI, Hutton et al., 2020). Hughes et al. (2021) implemented BMI for MODFLOW 6 (MF6) and extended it to handle numerical-level model couplings. Hughes et al. (2021) specifically demonstrated the ease of coupling the PRMS model to MF6's BMI interface. 

This notebook reproduces the results of Hughes et al. (2021) but substitutes `pywatershed` for [PRMS-BMI](https://github.com/nhm-usgs/bmi-prms-demo) in the Sagehen Creek Watershed Simulation. The pywatershed package contains the same core functionality as the PRMS model in an interactive language with a more modularized design. The goal of this notebook is to give a concrete example of how pywatershed can couple to MF6 or to other models via BMI.

## Model coupling design

The model coupling design of Hughes et al. (2021) is similar to GSFLOW, with the following differences:
* Surface water is modeled on HRUs instead of a grid.
* The surface water soilzone process is only solved once per timestep instead of being recalculated with MF6 outer solver iterations.

The coupling in in this notebook is more like GSFLOW and different from Hughes et al. (2021) in that:
* Surface runoff and interflow are routed from upslope to downslope HRUs using "cascading flow".

Figure 1 below is a GSFLOW schematic from Markstrom et al. (2008). It shows two conceptual soil models interacting in this design, separated by the dashed line. Above the dashed line is the PRMS "soilzone" which takes infiltration terms (ssres_infil + pref_flow_infil) from PRMS internally. Vertical downward fluxes out of this PRMS soilzone (ssr_to_gw + soil_to_gw) which normally flow to the PRMS conceptual groundwater model are instead sent through the API to the unsaturated-zone flow (UZF) package of MF6 which show below the PRMS soilzone in the figure. (This flux is not explicit in the figure). Surface and subsurface "cascading" or lateral flow terms (hru_horton_cascflow + dunnianflow + pref_flow + slow_flow, represented by Hortonian runoff, Dunnian runoff, and interflow in the figure) from PRMS are mapped to the stream flow routing (SFR) package of MF6. Finally, evaporation from the saturated root zone is calculated by the UZF package in MF6 and we can pass unused potential evapotranspiration (`unused_potet`) from PRMS through the API to UZF.

|![GSFlow schematic](gsflow_schematic.png)|
|:--:|
| <i><b>Figure 1.</b> GSFLOW conceptual schematic diagram from Markstrom et al. (2008) </i>|

Figure 2 shows additional detail on how pywatershed and MF6 are one-way coupled in this notebook. Pywatershed runs the hydrologic simulation on an unstructured grid of hydrologic response units (HRUs) as shown on the top. Starting from atmospheric focing inputs, and proceeding through canopy, snow, runoff, and soil process representations the same as used by GSFLOW except these are on HRUs in this notebooks. At the end of each daily timestep, fluxes and states are sent from pywatershed to MF6 using the MODFLOW6 API. As shown by arrows in the figure, separate spatial weights map states and fluxes from the HRUs to the MODFLOW grid and streamflow network. The gridded MF6 Unsaturated Zone Flow (UZF) package is conceptualized below the soilzone of pywatershed model and receives its vertical outflows. Unstatisfied potentital evapotransipiration in the hydrologic model, remapped from HRUs to gridcells, can be met by available water in the UZF rootzone. The hydrologic Hortonian, Dunnian and interflow runoff fluxes are mapped from the HRUS into the MF6 streamflow routing (SFR) network.

Also shown in the figure, the model design allows the MF6 Unstaurated Zone Flow (UZF) package to determine saturation and reject some or all of its infiltration term coming from the PRMS soilzone. Instead of sending these back in a 2- way coupling, the rejected water is sent as runoff to streamflow routing (SFR) using the "mover" (MVR) package.  Similarly, the MODFLOW drain (DRN) Package was used to simulate groundwater exfiltration to the land surface in areas where the groundwater levels exceed land surface. This exfiltration is mapped as runoff to SFR via MVR.

| ![sagehen_schematic](sagehen_schematic_pws.png) |
|:--:|
| <i><b>Figure 2.</b> Overview of spatial discretizations, the pywatershed-MF6 coupling, and physical process representations used for modeling the Sagehen Creek Watershed. The execution loop first executes pywatershed (1), then its fluxes are disaggregated with the two mappings and sent to MF6 (2) for execution via its BMI API. </i>|

In [ ]:
import os
import pathlib as pl
import platform
import shutil

import flopy
import hvplot.xarray  # noqa
import jupyter_black
import numpy as np
import pywatershed as pws
import xarray as xr
from modflowapi import ModflowApi

jupyter_black.load()
pws.utils.gis_files.download()
pws.utils.addtl_domain_files.download()

nb_output_dir = pl.Path("./step3_mf6_api").resolve()
if not nb_output_dir.exists():
    nb_output_dir.mkdir()

pws_root = pws.constants.__pywatershed_root__
domain_dir = (pws_root.parent / "test_data/sagehen_5yr/").resolve()
mf6_domain_dir = (pws_root.parent / "test_data/sagehen_mf6").resolve()

## Get the MF6 dylibs or dll from the nightly build
Keep it fresh.

In [ ]:
nightly_build_dir = nb_output_dir / "mf6_nightly_build"
nightly_build_dir.mkdir(exist_ok=True)
flopy.utils.get_modflow(
    bindir=str(nightly_build_dir), repo="modflow6-nightly-build", quiet=True
)

if platform.system() == "Windows":
    mf6_dll = nightly_build_dir / "libmf6.dll"
elif platform.system() == "Darwin":
    mf6_dll = nightly_build_dir / "libmf6.dylib"
else:
    mf6_dll = nightly_build_dir / "libmf6.so"

assert mf6_dll.exists()

## MF6 setup
The MF6 input files need copied to a run directory. One should look at `run_dir/mfsim.nam` (copied from the source `sagehenmodel` directory) to inspect the MF6 model setup up in more detail. Looking at `run_dir/ex-gwf-sagehen-gsf.tdis` we can see that the time units are days. The length units of meters are not explicitly stated, but we know that the grid in the `common/gwf_sagehen-gsf.dis` are in meters. We can also see that output will be sent to a directory `output` within the run directory. If we dont create this, the dylib/dll will die when it cant output to that location. 

In [ ]:
files_dirs_to_cp = [
    "common",
    "sagehenmodel",
    "hru_weights.npz",
    "sagehen_postprocess_graphs.py",
    "prms_grid_v3-Copy1.nc",
]
rename = {
    "sagehenmodel": "run_dir",
    "sagehen_postprocess_graphs.py": "run_dir/sagehen_postprocess_graphs.py",
}
for name in files_dirs_to_cp:
    src = mf6_domain_dir / name
    if name in rename.keys():
        dst = nb_output_dir / rename[name]
    else:
        dst = nb_output_dir / name
    if not dst.exists():
        if src.is_dir():
            shutil.copytree(src, dst)
        else:
            shutil.copy(src, dst)

In [ ]:
run_dir = nb_output_dir / "run_dir"
assert run_dir.exists()

mf6_output_dir = run_dir / "output"
# This step is *critical* for MF6 initialization
if not mf6_output_dir.exists():
    mf6_output_dir.mkdir()

## Pywatershed setup
The pyawatershed files dont need copied, they are just used in memory. 

In [ ]:
control_file = domain_dir / "sagehen_no_gw_cascades.control"
control = pws.Control.load_prms(control_file)

parameter_file = domain_dir / control.options["parameter_file"]
params = pws.parameters.PrmsParameters.load(parameter_file)
params = pws.utils.preprocess_cascades.preprocess_cascade_params(
    control, params, verbosity=0
)

We'll do some customization of the control

In [ ]:
# use the 16 year CBH files from supplementary pywatershed files
control.options["input_dir"] = (
    pws.constants.__pywatershed_root__
    / "data/pywatershed_addtl_domains/sagehen_cbh_16yr"
)
control.edit_n_time_steps(5844)  # the 16 years in the forcing files above
control.options["calc_method"] = "numba"
# Runoff and Soil cascades fail mass balance at some low level though their
# outputs match PRMS, so we'll turn off the budget for now.
control.options["budget_type"] = None
control.options["netcdf_output_dir"] = nb_output_dir / "pws_model_output"

We discussed the coupling strategy above, but now it's time to get specific with pywatershed. Fortunately, we can ask pywatershed for details about its fluxes and states. For fluxes leaving the model, we can look specifically at the "output" mass balance terms. 

In [ ]:
pws.meta.get_vars(
    pws.PRMSRunoffCascadesNoDprst.get_mass_budget_terms()["outputs"]
)

From the `PRMSRunoffCascadesNoDprst` process, the `hru_horton_cascflow` variable is one component of flux we want to map to SFR in MF6. Looking at `PRMSSoilzoneCascadesNoDprst` next, 

In [ ]:
pws.meta.get_vars(
    pws.PRMSSoilzoneCascadesNoDprst.get_mass_budget_terms()["outputs"]
)

To SFR, we'll also want to add `dunnian_flow`, `pref_flow`, and `slow_flow`. Note that all these fluxes to SFR are reported by PRMS in inches on an HRU per day. We'll need to convert these depths into volumes in cubic meters for SFR. This will require the area of the HRUs from which these fluxes come. 

Similarly, the vertical fluxes from `PRMSSoilzoneCascadesNoDprst` to UZF, `soil_to_gw` and `ssr_to_gw`, are in inches on an HRU per day and we'll have to make the same unit conversion to volume for MF6 inputs. Finally, let's look at `unused_potet`. We'll verify that it's a varible in `PRMSSoilzoneCascadesNoDprst`, which is the last process in pywatershed to take its toll on potential evapotranspiration.

In [ ]:
assert "unused_potet" in pws.PRMSSoilzoneCascadesNoDprst.get_variables()
pws.meta.get_vars("unused_potet")

Again, the state of this variable is in inches on an HRU and will need the same conversion as the other variables.

We specify the variable names we'd like to be output by the pywatershed model run. We make this output optional with the `pws_write_output` variable. Below, we'll actually write our own custom output routine for pywatershed to see how easy that is.

In [ ]:
pws_write_output = False
output_var_names = [
    "hru_ppt",
    "potet",
    "hru_actet",
    "ssres_in",
    "pref_flow_infil",
    "ssr_to_gw",
    "soil_to_gw",
    "sroff",
    "ssres_flow",
    "pref_flow",
    "dunnian_flow",
    "slow_flow",
    "hru_horton_casscflow",
    "dunnian_flow",
]
control.options["netcdf_output_var_names"] = output_var_names

We can specify and initialize the model here

In [ ]:
prms_processes = [
    pws.PRMSSolarGeometry,
    pws.PRMSAtmosphere,
    pws.PRMSCanopy,
    pws.PRMSSnow,
    pws.PRMSRunoffCascadesNoDprst,
    pws.PRMSSoilzoneCascadesNoDprst,
]

prms_model = pws.Model(
    prms_processes,
    control=control,
    parameters=params,
)

## Spatial mappings
These mappings from HRUs to the MF6 grid and to the SFR network are magically created. 

In [ ]:
weights = np.load(nb_output_dir / "hru_weights.npz")

_HRU to UZF weights_

In [ ]:
uzfw = weights["uzfw"]
nuzf_infilt = uzfw.shape[0]
uzfw.shape

_HRU to SFR weights_

In [ ]:
sfrw = weights["sfrw"]
sfrw.shape

This indicates that here are 128 HRUs, 3386 gridcells, and 201 stream reaches. 

Define a function to map HRU values to MODFLOW 6 values, which is just a dot product.

In [ ]:
def hru2mf6(weights, values):
    return weights.dot(values)

## Unit conversion factors
As noted above, we'll need to comvert from inches on an hru to cubic meters. The HRU areas also happen to be in acres! This is FUN.

In [ ]:
m2ft = 3.28081
in2m = 1.0 / (12.0 * m2ft)
acre2m2 = 43560.0 / (m2ft * m2ft)
hru_area_m2 = params.parameters["hru_area"] * acre2m2

## Run one-way coupled pywatershed and MODFLOW 6 models


In [ ]:
os.chdir(run_dir)
print("changing to run directory:\n", os.getcwd())
ntimes = int(control.n_times)
print("Number of days to simulate {}".format(ntimes))

### Custom pywatershed output for figures
While pywatershed has NetCDF output available, the existing plots are setup to work with data collected during the run saved to NetCDF at the end of the run. We follow this path for ease of reproducing the plots and this illustrates how custom output can be quite easily achieved with pywatershed. Custom output which saves data to memory and only writes after the simulation, as done below, can signficantly decrease pywatershed run times as writing to netcdf incurs a wall time cost with output to disk during the model run loop. 

In [ ]:
# We'll store a dictionary of numpy arrays and convert to netcdf via
# xarray on output.
prms_vars = [
    "hru_ppt_out",
    "actet_out",
    "unused_potet_out",
    "prms_infil_out",
    "runoff_out",
    "interflow_out",
]
prms_var_dict = {}
prms_var_dict["time_out"] = np.empty(ntimes, dtype="datetime64[s]")
for vv in prms_vars:
    prms_var_dict[vv] = np.zeros(
        (ntimes, hru_area_m2.shape[0]), dtype=np.float64
    )

### Initialize MODFLOW 6

The dylib/DLL Initialization requires all inputs AND the output directory to exist.

In [ ]:
mf6_config_file = "mfsim.nam"
mf6 = ModflowApi(mf6_dll, working_directory=os.getcwd())
mf6.initialize(str(mf6_config_file))

Get information about the modflow model start and end times

In [ ]:
current_time = mf6.get_current_time()
end_time = mf6.get_end_time()
print(
    f"MF current_time: {current_time}, prms control.start_time: {control.start_time}"
)
print(f"MF end_time: {end_time}, prms control.n_times: {control.n_times}")

### Get pointers to MODFLOW 6 variables

As shown in Figures 1 and 2, pywatershed is sending the following fluxes to MF6

* SINF is surface infiltration to MF6's UZF package, this is mapped from groundwater recharge in pywatershed
* PET is unsatisfied potential evapotransipiration in MF6's UZF, this is also calculated in the soilzone in pytwatershed
* RUNOFF to MF6's SFR comes from combined surface and subsurface runoff from pywatershed.

To set these values on MF6, we need the to get the pointers into MF6 using its BMI interface.

In [ ]:
sim_name = "sagehenmodel"
mf6_var_model_dict = {"SINF": "UZF-1", "PET": "UZF-1", "RUNOFF": "SFR-1"}
mf6_vars = {}
for vv, mm in mf6_var_model_dict.items():
    mf6_vars[vv] = mf6.get_value_ptr(
        mf6.get_var_address(vv, sim_name.upper(), mm)
    )

for vv, dd in mf6_vars.items():
    print(f"shape of {vv}: {dd.shape}")

The UZF model has 2 vertical layers, so the number of points in UZF is twice what was shown in the spatial weights mappings above. The first layer is first in the linear dimension.

### Run the models

Now we run the model. We use pywatershed to control the simulation.

In [ ]:
if pws_write_output:
    prms_model.initialize_netcdf()

n_time_steps = control.n_times  # redundant above?
for istep in range(n_time_steps):
    prms_model.advance()

    if control.current_dowy == 1:
        if istep > 0:
            print("\n")
        print(f"Water year: {control.current_year + 1}")

    msg = f"Day of water year: {str(control.current_dowy).zfill(3)}"
    print(msg, end="\r")

    # run pywatershed
    prms_model.calculate()
    prms_model.output()  # if output is not initialized, does nothing

    # calculate variables for output and coupling. Some variables output just for
    # diagnostic purposes
    hru_ppt = prms_model.processes["PRMSAtmosphere"].hru_ppt.current
    prms_infil = (
        prms_model.processes["PRMSSoilzoneCascadesNoDprst"].ssres_in
        + prms_model.processes["PRMSSoilzoneCascadesNoDprst"].pref_flow_infil
    )
    actet = prms_model.processes["PRMSSoilzoneCascadesNoDprst"].hru_actet

    # coupling variables
    unused_potet = prms_model.processes[
        "PRMSSoilzoneCascadesNoDprst"
    ].unused_potet

    uzf_recharge = (
        prms_model.processes["PRMSSoilzoneCascadesNoDprst"].ssr_to_gw
        + prms_model.processes["PRMSSoilzoneCascadesNoDprst"].soil_to_gw
    )

    hortonian = prms_model.processes[
        "PRMSRunoffCascadesNoDprst"
    ].hru_horton_cascflow
    dunnian = prms_model.processes["PRMSSoilzoneCascadesNoDprst"].dunnian_flow
    pref_flow = prms_model.processes["PRMSSoilzoneCascadesNoDprst"].pref_flow
    slow_flow = prms_model.processes["PRMSSoilzoneCascadesNoDprst"].slow_flow
    prms_ro = (
        (hortonian + dunnian + pref_flow + slow_flow) * in2m * hru_area_m2
    )

    # save PRMS results (converted to m3(/d))
    prms_var_dict["time_out"][istep] = control.current_time
    prms_var_dict["hru_ppt_out"][istep, :] = hru_ppt * in2m * hru_area_m2
    prms_var_dict["unused_potet_out"][istep, :] = (
        unused_potet * in2m * hru_area_m2
    )
    prms_var_dict["actet_out"][istep, :] = actet * in2m * hru_area_m2
    prms_var_dict["prms_infil_out"][istep, :] = prms_infil * in2m * hru_area_m2
    prms_var_dict["runoff_out"][istep, :] = (
        (hortonian + dunnian) * in2m * hru_area_m2
    )
    prms_var_dict["interflow_out"][istep, :] = (
        (pref_flow + slow_flow) * in2m * hru_area_m2
    )

    # Set MF6 pointers
    mf6_vars["RUNOFF"][:] = hru2mf6(sfrw, prms_ro)  # sroff + ssres_flow
    mf6_vars["SINF"][:nuzf_infilt] = (
        hru2mf6(uzfw, uzf_recharge) * in2m
    )  # ssr_to_gw + soil_to_gw
    mf6_vars["PET"][:nuzf_infilt] = hru2mf6(uzfw, unused_potet) * in2m

    # run MODFLOW 6
    mf6.update()

try:
    mf6.finalize()
    prms_model.finalize()
    success = True
except:
    raise RuntimeError

### Save custom pywatershed output to NetCDF
Variables collected in memory from pywatershed are converted to NetCDF via xarray data sets. We choose to write one file per variable.

In [ ]:
pws_output_dir = nb_output_dir / "pws_output"
pws_output_dir.mkdir(exist_ok=True)
for prms_var in prms_vars:
    if prms_var == "time_out":
        continue
    var = prms_var[0:-4]
    print(var)
    ds = xr.Dataset(
        data_vars={var: (["time", "nhru"], prms_var_dict[prms_var])},
        coords={"time": prms_var_dict["time_out"].astype("datetime64[ns]")},
    )
    ds.to_netcdf(pws_output_dir / f"{var}.nc")
    del ds

## Plot

We spare the user the details of the plotting. 

In [ ]:
fig_out_dir = nb_output_dir / "figures"
fig_out_dir.mkdir(exist_ok=True)

run_dir = nb_output_dir / "run_dir"
os.chdir(run_dir)  # make sure

import sagehen_postprocess_graphs as graphs

In [ ]:
graphs.streamflow_fig()

In [ ]:
graphs.et_recharge_ppt_fig()

In [ ]:
graphs.gwf_uzf_storage_changes_fig()

In [ ]:
graphs.cumulative_streamflow_fig()

In [ ]:
graphs.composite_fig()

Compare our plots side-by-side, very scientific! Remember that we are not using the exact same model coupling design, we have introduced cascading flow to the model use by Hughes et al. (2022). So the results wont match, but they should be close. 
<div id="tablediv">
  <table width="100%">
    <tr>
        <th>Our results</th>
        <th>Hughes et al. (2022), Figure 12</th>
      </tr>
      <tr>
      <td><img src="step3_mf6_api/figures/sagehen_pywatershed_graphs.png" alt="Our results"></td>
      <td><img src="hughes_et_al_sagehen_3panel.jpg" alt="Hughes et al."></td>        
    </tr>
  </table>
</div>


## Conclusions

This one-way coupling of pywatershed and MODFLOW 6 demonstrates how software interoperatbility standards can bridge hydrologic models from "different" domains. 


Pywatershed is close to reproducing GSFLOW, with "cascading flows" and 2-way couling with MF6 to be available in version 3.0.

## References
* Hutton, E. W., Piper, M. D., & Tucker, G. E. (2020). The Basic Model Interface 2.0: A standard interface for coupling numerical models in the geosciences. Journal of Open Source Software, 5(51), 2317.
* Hughes, J. D., Russcher, M. J., Langevin, C. D., Morway, E. D., & McDonald, R. R. (2022). The MODFLOW Application Programming Interface for simulation control and software interoperability. Environmental Modelling & Software, 148, 105257.
* Langevin, C. D., Hughes, J. D., Banta, E. R., Niswonger, R. G., Panday, S., & Provost, A. M. (2017). Documentation for the MODFLOW 6 groundwater flow model (No. 6-A55). US Geological Survey.
* Markstrom, S. L., Niswonger, R. G., Regan, R. S., Prudic, D. E., & Barlow, P. M. (2008). GSFLOW-Coupled Ground-water and Surface-water FLOW model based on the integration of the Precipitation-Runoff Modeling System (PRMS) and the Modular Ground-Water Flow Model (MODFLOW-2005). US Geological Survey techniques and methods, 6, 240.
* Regan, R. S., Markstrom, S. L., Hay, L. E., Viger, R. J., Norton, P. A., Driscoll, J. M., & LaFontaine, J. H. (2018). Description of the national hydrologic model for use with the precipitation-runoff modeling system (prms) (No. 6-B9). US Geological Survey.
* Regan, R.S., Markstrom, S.L., LaFontaine, J.H., 2022, PRMS version 5.2.1: Precipitation-Runoff Modeling System (PRMS): U.S. Geological Survey Software Release, 02/10/2022.